# Dinamik Uçuşlar Haritası

Python programlama dili kullanarak dinamik bir uçuşlar haritası oluşturmak için aşağıdaki kütüphaneleri ekliyoruz.

- requests : Verileri API üzerinden almak için kullanılır.
- pandas : Veri işlemesi ve analizi için kullanılır.
- numpy : Çok boyutlu dizileri, matrisleri destekleyen, bu diziler üzerinde çalışacak üst düzey matematiksel işlevler ekleyen bir kütüphanedir.
- bokeh : Veri görselleştirmesi için kullanılan kütüphanedir.

In [ ]:
import requests
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import LabelSet, ColumnDataSource, DataTable,  TableColumn, Label,Panel, Tabs, CDSView, BooleanFilter
from bokeh.tile_providers import get_provider, OSM
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import layout

# output_notebook() çıktıları notebook üzerinde göstermemize yarar.
output_notebook()

Loading BokehJS ...

Aşağıda veri çekme, düzenleme ve görselleştirme için kullandığım fonksiyonlar bulunmaktadır.
- Bu iki fonksiyon API' den gelen veride bulunan koordinat bilgilerini dönüştürmemize yarar.
- Veriler coğrafi koordinat olarak gelmektedir.
- Ama bokeh ile gösterebilmemiz için web mercatora dönüştürmemiz gerekir.
- Bu dönüşüm işlemlerini aşağıdaki iki fonksiyonu kullanarak yapıyoruz.
- İlk fonksiyon nokta bazlı dönüşüm, ikinci fonksiyon ise tüm noktaları dönüştürür.

In [2]:
#Nokta koordinatları dönüştürmek için fonksiyon
def wgs84_web_mercator_point(lon,lat):
    k = 6378137
    x= lon * (k * np.pi/180.0)
    
    y= np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x,y

#Data içindeki koordinatları dönüştürmek için fonksiyon
def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["X"] = df[lon] * (k * np.pi/180.0)
    df["Y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

- df_format fonksiyonu veriyi analiz etmek ve görselleştirmek için uygun formata getirilmesini sağlar.
- Fonksiyon API' den gelen yanıtı alır. Düzgün bir formata getirir.

In [3]:
#Requestle çekilen veriyi DataFrame kullanarak uygun formata getirmek için fonksiyon
def df_format(response):
    
    #Sütun isimleri 
    col_name = ['icao24','callsign','origin_country','time_position','last_contact',
            'long','lat','baro_altitude','on_ground','velocity',       
            'true_track','vertical_rate','sensors','geo_altitude','squawk','spi',
            'position_source']
    
    #Pandas.DataFrame kullanılarak veri düzenlendi
    df = pd.DataFrame(response) #Yanıt içindeki "states" df ye tanımlandı
    df = df.loc[:,0:16] #df tümsatırları alındı ve 17 sütuna ayrıldı
    df.columns = col_name #Tanımlanan sütun isimleri eklendi
    wgs84_to_web_mercator(df,"long","lat") #wgs84 ten mercator dönüşümü yapıldı. X ve Y sütunları
    df = df.fillna(-1) #NaN değerleri No Data olarak değiştirildi.
    df["rot_angle"] = (df["true_track"]*-1)+45 #icon açısı için rot_angle oluşturuldu.
    df["url"] = icon_url #icon tanımlandı
    list_str = []
    for i in df["baro_altitude"]:
        list_str.append(str(i)+" m")
    df["str_altitude"] = list_str
    return df

- country_plane() fonksiyonu ülkeler ve kaç uçaklarının olduğunu gösteren grafik için bir dataframe oluşturur.

In [4]:
#Grafik oluşturmak için ülke isimleri ve hangi ülkenin kaç uçağı olduğunu gösteren bir DataFrame oluşturan fonksiyon
def country_plane(data,country):
    
    countrys = list(set(data[country]))
    planes = []
    for i in countrys: 
        counter = 0 
        for j in data[country]:
            if i == j:
                counter+=1
        planes.append(counter)
    
    countrys.append("Toplam")
    planes.append(sum(planes))
    df = pd.DataFrame({"Country":countrys,"planes":planes})
    df.sort_values("planes",ascending=False)
    return df

- Yüksekliklere göre filtrelemek için oluşturulan fonksiyon.

In [5]:
def altitude_filter(source,min_altitude,max_altitude):
    alt_filter = [True if int(alti) > min_altitude and int(alti) <= max_altitude else False for alti in source.data["baro_altitude"]]
    return alt_filter

- Daire ikonu oluşturmak için kullanılan fonksiyon.

In [6]:
def create_circle(p,size,color,source,view,legend_label):
    p.circle(x="X",y="Y",size=size,fill_color=color,line_color=color,legend_label=legend_label,view=view,fill_alpha=0.6,line_width=3,source=source) 

- Gerçek zamanlı bir harita oluşturmak için aşağıdaki fonksiyonu kullanıldı.
- Stream verileri kullanmak için CollumnDataSource(CDS) oluşturuldu.
- flights_map fonksiyonu içerisinde update() fonksiyonu oluşturuldu.
- update() fonksiyonunu kullanarak API'den verileri çekip, daha önce yazdığımız fonksiyonları kullandık.
- Harita üzerinde ülkelerin bir listeni oluşturmak için toText fonksiyonunu kullandık.
- CDS nesnesini CDS.stream() kullanarak güncelliyoruz.
- doc.add_periodic_callback(update,5000) komutu update fonksiyonunu 5000 ms aralıklarla tekrar çalıştırmamıza yarar.
- Bu sayade verilerimiz 5 saniyede bir güncellenmiş olur.
- Verileri harita üzerinde göstermek için bir figür oluştururuz.
- Altlık olarak OpenStreetMap kullanırız.

In [7]:
#Gerçek zamanlı bir harita oluşturmak verilerini sürekli güncellemek için fonksiyon oluşturuldu.    
def flights_map(doc):
    
    #Veri kaynağı parametrelerini düzgün bir şekilde almak için ColumnDataSource kullanıldı.
    url_source = ColumnDataSource({
        'icao24':[],'callsign':[],'origin_country':[],
        'time_position':[],'last_contact':[],'long':[],'lat':[],
        'baro_altitude':[],'str_altitude':[],'on_ground':[],'velocity':[],'true_track':[],
        'vertical_rate':[],'sensors':[],'geo_altitude':[],'squawk':[],'spi':[],
        'position_source':[],'X':[],'Y':[],'rot_angle':[],'url':[] })
    
    flight_source = ColumnDataSource({"Country":[],"planes":[],"text":[],"x":[],"y":[]})
    
    #ColumnDataSource verilerini güncellemek için fonksiyon
    def update():
        
        response = requests.get(url).json() 
        response = response["states"]      
        url_data = df_format(response) #Uçuş haritası için gerekli format
        flight_df = country_plane(url_data,"origin_country") #Grafik için gerekli format           
        def toText(df,col1,col2):
            list1=[]
            list2=[]
            text_list=[]
            for i in df[col1]:
                list1.append(i)
            for i in df[col2]:
                list2.append(i)   

            i=0
            while i < len(df[col1]):
                text = "{} : {}".format(list1[i],list2[i])
                text_list.append(text)
                i+=1
            return text_list 
        
        flight_df["text"] = toText(flight_df,"Country","planes")
        Y_SCALE = pd.DataFrame(list(range(0,len(flight_df["Country"]))))*19
        
        flight_df["x"] = X_SCALE
        flight_df["y"] = Y_SCALE 
        
        #flight_source verilerini güncellemek için .stream kullanılıyor. Veri kümesi içinde sadece değişen hücreleri güncelliyor.
        n_roll=len(flight_df.index)
        flight_source.stream(flight_df.to_dict(orient="list"),n_roll)
        
        #url_source verilerini güncellemek için .stream kullanılıyor
        n_roll=len(url_data.index)
        url_source.stream(url_data.to_dict(orient="list"),n_roll)
        
    #update fonksiyonunu 5 saniye aralıklarla çalıştırıyor.
    doc.add_periodic_callback(update,5000)
    
    
    #Harita için Figure oluşturuluyor.
    p_map = figure(plot_width = FIGURE_W, plot_height = FIGURE_H,
            x_range=x_range,y_range=y_range,
            x_axis_type="mercator",y_axis_type="mercator", sizing_mode="scale_width",
            tooltips=[("Call Sign","@callsign"),("Country","@origin_country"),("Barometric Altitude", "@str_altitude")]
            )

    # Uçak ikonu ekleniyor.
    p_map.image_url(url="url", x="X", y="Y", source=url_source,
                anchor="center", angle_units="deg",angle="rot_angle",
                h_units="screen",w_units="screen",h=IMAGE_H,w=IMAGE_W)
    p_map.circle(x="X",y="Y",size = 10,fill_color="red",line_color="red",
                fill_alpha = 0,line_width=0,source=url_source)

    #Etiket olarak uçak çağrı adları tanımlanıyor.
    labels = LabelSet(x = "X", y = "Y", text ="callsign", level="glyph",  
                x_offset=2, y_offset=2, source=url_source, render_mode='canvas',
                background_fill_color='white',text_font_size="5pt")
    
    
    country_label = LabelSet(x="x", y="y", x_units='screen', y_units='screen',
                 text="text", render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 source=flight_source)

    osmHarita = get_provider(OSM) #Altlık 
    p_map.add_tile(osmHarita) #Altlık ekleme
    p_map.add_layout(labels)  #Çağrı adları yazan label ekleniyor   
    p_map.add_layout(country_label)
    p_map.title = " TÜRKİYE UÇUŞLAR HARİTASI " #Başlık
    tab1 = Panel(child=p_map,title="Map")
    
    flight_range = flight_source.data["Country"] #Veri bar grafiği çizdirmek için belirlenmesi gereken y sınırı. !!!!!!!Grafik çizilememe hatası burasıyla ilgili olabilir!!!!!!!!)
    
    #Veri tablosu oluşturuluyor
    columns = [TableColumn(field = "Country", title="Countrys"),TableColumn(field="planes",title="Planes")] #Veri tablosunun sütun isimleri 
    data_table = DataTable(source=flight_source, columns=columns)
    tab2 = Panel(child=data_table,title="Data Table")
    
    tab = Tabs(tabs=[tab1,tab2]) #İlk sekmede dinamik harita, ikici sekme data table
    doc.add_root(tab)

- Open Sky Network hesap bilgileri girildi.
- Uçuş sınırları, harita boyutlar, ikon boyutları vb. bilgileri girildi.
- Verileri alıcağımız API linkini tanımlandı.
- Koordinat dönüşümleri yapıldı.
- Ve flights_map fonksiyonu çalıştırılarak Türkiye Uçuşlar Dinamik haritası oluşturuldu.

In [11]:
#API için kullanıcı adı ve şifre, çalışma alanı sınırları - Ücretsiz OpenSkyNetwork hesabı 
user_name = "*****"
password = "*****"

#Kod içinde sınır için kullanılan sayısal veriler.
#Uçuş sınırları
lon_min,lat_min = 25,35 
lon_max,lat_max = 45,45
#Harita boyutları
FIGURE_W = 400
FIGURE_H = 200
#Uçuş verilerinin yazdırıldığı katmanın x koordinat değeri
X_SCALE = 800
#İkon boyutları
IMAGE_H = 10
IMAGE_W = 10

#https://opensky-network.org/api/states/all?lamin={}&lomin={}&lamax={}&lomax={}
url = "https://{}:{}@opensky-network.org/api/states/all?lamin={}&lomin={}&lamax={}&lomax={}".format(user_name,password,lat_min,lon_min,lat_max,lon_max)
icon_url = "https://iconvulture.com/wp-content/uploads/2017/12/plane.svg?1648292982753" 

#Nokta koordinat dönüşümü icon için
xy_min=wgs84_web_mercator_point(lon_min,lat_min)
xy_max=wgs84_web_mercator_point(lon_max,lat_max)

x_range = [xy_min[0],xy_max[0]]
y_range = [xy_min[1],xy_max[1]]


show(flights_map)

- Uçuşlar için yükseklik filtrelemesi yapıldı.

In [16]:
#API için kullanıcı adı ve şifre, çalışma alanı sınırları
user_name = "*****"
password = "*****"
lon_min,lat_min = 25,35
lon_max,lat_max = 45,45

url = "https://{}:{}@opensky-network.org/api/states/all?lamin={}&lomin={}&lamax={}&lomax={}".format(user_name,password,lat_min,lon_min,lat_max,lon_max) #Api linki
icon_url =  "https://iconvulture.com/wp-content/uploads/2017/12/plane.svg?1648292982753" #Kullanılacak ikonun linki

response = requests.get(url).json() #Api
states = response["states"] 
url_data = df_format(states) #Veriyi istediğimiz Dataframe formatına çevirdik.
url_data.head()
url_source = ColumnDataSource(url_data)

p = figure(plot_width = FIGURE_W, plot_height = FIGURE_H,
            x_range=x_range,y_range=y_range,
            x_axis_type="mercator",y_axis_type="mercator", sizing_mode="scale_width",
            tooltips=[("Call Sign","@callsign"),("Country","@origin_country"),("Barometric Altitude", "@str_altitude")])

alti_list = [0, 2500, 5000, 7500, 10000, 12500]
color_list = ["aqua","blue","green","yellow","orange","red"]
for i in range(len(alti_list)):
    alti_filter = altitude_filter(url_source, alti_list[i], alti_list[i] + 2500)
    view = CDSView(source = url_source,filters = [BooleanFilter(alti_filter)])
    create_circle(p, 4, color_list[i], url_source, view, f"{alti_list[i]}-{alti_list[i]+2500} m")


osmHarita = get_provider(OSM) #Altlık 
p.add_tile(osmHarita)    
    
show(p)

- Uçuşların ülkelere göre grafiği oluşturuldu.

In [17]:
#API için kullanıcı adı ve şifre, çalışma alanı sınırları
user_name = "*****"
password = "*****"
lon_min,lat_min = 25,35
lon_max,lat_max = 45,45

url = "https://{}:{}@opensky-network.org/api/states/all?lamin={}&lomin={}&lamax={}&lomax={}".format(user_name,password,lat_min,lon_min,lat_max,lon_max) #Api linki
icon_url =  "https://iconvulture.com/wp-content/uploads/2017/12/plane.svg?1648292982753" #Kullanılacak ikonun linki

#Grafik için Figure oluşturuluyor.
flight_source = ColumnDataSource({"Country":[],"planes":[]})
y_range = flight_source.data["Country"]

response = requests.get(url).json() #Api
states = response["states"]
url_data = df_format(states)
flight_df = country_plane(url_data,"origin_country")

n = len(flight_df.index)
flight_source.stream(flight_df.to_dict(orient="list"),n)


p = figure(title="Ülkeler ve Uçak Sayıları", y_range =y_range, x_axis_label="UÇAK SAYISI", y_axis_label="ÜLKELER",tooltips=[("","@planes")])
p.hbar(y="Country",left=0,right="planes",color="orange",height= 0.6,fill_alpha=0.8 ,source=flight_source)
         
show(p)